In [4]:
from sklearn.model_selection import train_test_split
from config import *
from utils import *

## Read and Process Data

In [2]:
X, y = get_data(TRAIN_PATH)
X.head(2)

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt
0,2,3.0,299,0,1,1,7,0,1,1,2,2,2,1,1.0,100.0,41326,0.0,1.0
1,2,1.0,265,0,1,1,2,0,2,2,3,3,3,1,1.0,0.0,41401,0.0,2.0


In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, train_scaler = normalization(X_train, numerical_features)
X_train.head(2)

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt
4625,1,-0.241626,307,0,2,1,2,5,2,1,2,2,2,1,-0.392801,-0.278996,41326,-0.165571,-0.823408
8508,2,-0.408906,299,0,2,1,6,0,1,1,2,1,2,1,-0.392801,-0.278996,41330,-0.165571,-0.252048


In [5]:
X_val = normalization(X_val, numerical_features, train_scaler)
X_val.head(2)

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt
9656,2,-0.464665,264,264,3,1,6,0,2,3,3,3,3,1,0.281102,-0.278996,41401,-0.165571,-0.252048
5220,2,-0.464665,285,266,3,7,0,0,2,2,2,2,2,1,0.281102,3.679910,41326,-0.165571,-0.252048
